In [19]:
# Import required Packages

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import pandas as pd

In [21]:
#start_url = 'https://www.udemy.com/course/aws-certified-developer-associate-dva-c01/'
start_url ='https://www.udemy.com/course/aws-certified-solutions-architect-associate-saa-c02/'
#course_name ='aws-developer-associate'
course_name ='aws-soultions-architect-associate' 

with webdriver.Firefox() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(start_url)

 

    time.sleep(10)

 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)
    #button = driver.find_elements_by_xpath("//button[@data-purpose='show-more']")
    #print(button)
    #button.click()
    #time.sleep(5)

    # And grab the page HTML source
    html_page = driver.page_source
    
    # Save a local Copy
    with open(course_name+'.html','w') as file:
        file.write(html_page)
          

In [22]:
source_page =''   
# Read the local saved copy
with open(course_name+'.html','r') as file:
    source_page = file.read()  


In [23]:
# Start Processing the file
# Parse the HTML with BeautifulSoup and create a soup object
soup = BeautifulSoup(source_page)

table_columns = []
table_columns.append('title')
table_columns.append('lectures')
table_columns.append('duration')

table_data = []
# Select course list
course_content = soup.select('div[class*="section--section--"]')  
for EachPart in course_content:
    row_data = []
    section_title = EachPart.select('div[class*="section--section-title--"]')[0]
    section_content = EachPart.select('div[class*="udlite-text-sm section--section-content--"]')[0]
    section_content = section_content.get_text(separator=" ", strip=True).split('•',1)[0]
    section_duration = EachPart.select('span')[0]
    row_data.append(section_title.text)
    row_data.append(section_content)
    row_data.append(section_duration.text)
    table_data.append(row_data)
print(table_data)

[['Introduction - AWS Certified Solutions Architect Associate', '8 lectures ', '15min'], ['Code & Slides Download', '1 lecture ', '1min'], ['AWS Fundamentals: IAM & EC2', '30 lectures ', '2hr 36min'], ['High Availability and Scalability: ELB & ASG', '13 lectures ', '1hr 30min'], ['EC2 Storage - EBS & EFS', '12 lectures ', '58min'], ['AWS Fundamentals: RDS + Aurora + ElastiCache', '10 lectures ', '1hr'], ['Route 53', '14 lectures ', '1hr 3min'], ['Classic Solutions Architecture Discussions', '7 lectures ', '41min'], ['Amazon S3 Introduction', '10 lectures ', '1hr 4min'], ['AWS CLI, SDK, IAM Roles & Policies', '11 lectures ', '44min'], ['Advanced Amazon S3 & Athena', '14 lectures ', '1hr 14min'], ['CloudFront & AWS Global Accelerator', '5 lectures ', '39min'], ['AWS Storage Extras', '7 lectures ', '27min'], ['Decoupling applications: SQS, SNS, Kinesis, Active MQ', '13 lectures ', '1hr 25min'], ['Serverless Overviews from a Solution Architect Perspective', '14 lectures ', '1hr 5min'], ['S

In [24]:
# Create a Pandas DataFrame
df = pd.DataFrame(table_data, columns=table_columns)
df.to_excel(course_name+".xlsx") 